# LDaCA Analysis: Notebook Driver

This notebook helps you run the LDaCA web app from a Jupyter environment.

What this notebook does:
- Starts the FastAPI backend and the frontend/nginx using the repo’s `start` script (non‑blocking)
- Opens the UI via the Jupyter proxy in a new tab
- Shows a quick workspace object to confirm the backend is live

Before you run:
- Data root: the backend defaults to `DATA_ROOT=data` under the repository. You can override it by setting the `DATA_ROOT` environment variable before import. The first code cell sets it explicitly when needed.

Run order:
1) Start services (backend + frontend) in background
2) Open the UI through the proxy
3) Inspect current workspace

# Start Backend and Frontend (Background)

This will launch the FastAPI backend and the frontend/nginx via the repository `start` script in the background, then keep the notebook interactive.

Notes:
- The backend uses `DATA_ROOT` for storage (defaults to `data`). Set it before import to change location
- If a server is already running on the same ports, restart the kernel or pick a different port

In [ ]:
import os
import subprocess
from IPython.display import display, Markdown

os.environ["DATA_ROOT"] = "./ldaca_web_app/backend/data"
from ldaca_web_app_backend import start_backend, start_frontend

# Spawn start in background so the notebook remains interactive
proc = subprocess.Popen(["/bin/bash", "start"])
task = start_backend(port=8001)

### Opening the UI via Jupyter proxy
- If your browser blocks pop‑ups, use the printed link instead

In [ ]:
from IPython.display import Javascript
import os

base = (
    os.environ.get("JUPYTERHUB_SERVICE_PREFIX")
    or os.environ.get("JUPYTER_SERVER_URL")
    or os.environ.get("JUPYTER_SERVER_ROOT")
    or ""
)
if base and not base.endswith("/"):
    base += "/"

url = f"{base}proxy/{3000}/"

display(Javascript(f"window.open('{url}', '_blank');"))
display(Markdown(f"If popup was blocked, click: [Open Frontend]({url})"))

### Inspecting the current workspace

- The snippet fetches the current workspace for the `root` user and displays its nodes
- User data and workspaces are stored under `DATA_ROOT/users/user_root/` by default
- If this is your first run, folders are created automatically when the backend starts

In [ ]:
from ldaca_web_app_backend import workspace_manager

ws = workspace_manager.get_current_workspace("root")
ws.nodes